# Interpretation: Feature Analysis

Analyze which features are preserved in robust tickets.

In [ ]:
import sys
sys.path.append('..')

import numpy as np
import matplotlib.pyplot as plt
import torch

from src.utils.config import load_config, get_device
from src.models.deep_hedging import DeepHedgingNetwork
from src.utils.visualization import plot_feature_importance

## Load Models

In [ ]:
config = load_config('../config.yaml')
device = get_device(config)

# Standard ticket
model_standard = DeepHedgingNetwork(config['model'])
try:
    model_standard.load_state_dict(torch.load('../experiments/pruning/sparsity_80/model.pt', map_location=device))
    mask_standard = torch.load('../experiments/pruning/sparsity_80/mask.pt')
except:
    print("Standard ticket not found")
    model_standard = None

# Robust ticket
model_robust = DeepHedgingNetwork(config['model'])
try:
    model_robust.load_state_dict(torch.load('../experiments/adversarial_training/pgd_retrain_40epochs/model.pt', map_location=device))
    mask_robust = torch.load('../experiments/adversarial_training/mask.pt')
except:
    print("Robust ticket not found")
    model_robust = None

## Feature Importance Analysis

In [ ]:
def compute_feature_importance(model, mask):
    """
    Compute feature importance from input layer weights
    """
    # Get first layer weights
    first_layer_name = list(mask.keys())[0]
    W1 = model.network[0].weight.data  # (512, 8)
    W1_masked = W1 * mask[first_layer_name]
    
    # Importance: sum of absolute weights per input feature
    importance = torch.abs(W1_masked).sum(dim=0).cpu().numpy()
    importance = importance / importance.sum()  # Normalize
    
    return importance

feature_names = [
    'log(S/K)',
    'Return',
    'sqrt(v)',
    'Delta_v',
    'Time',
    'delta_prev',
    'Trade_vol',
    'PnL'
]

if model_standard is not None:
    importance_standard = compute_feature_importance(model_standard, mask_standard)
    print("Standard Ticket Feature Importance:")
    for name, imp in zip(feature_names, importance_standard):
        print(f"  {name}: {imp:.4f}")

if model_robust is not None:
    importance_robust = compute_feature_importance(model_robust, mask_robust)
    print("\nRobust Ticket Feature Importance:")
    for name, imp in zip(feature_names, importance_robust):
        print(f"  {name}: {imp:.4f}")

## Visualization

In [ ]:
if model_standard is not None and model_robust is not None:
    x = np.arange(len(feature_names))
    width = 0.35
    
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.bar(x - width/2, importance_standard, width, label='Standard Ticket', color='blue', alpha=0.8)
    ax.bar(x + width/2, importance_robust, width, label='Robust Ticket', color='red', alpha=0.8)
    
    ax.set_xlabel('Feature')
    ax.set_ylabel('Importance (normalized)')
    ax.set_title('Feature Importance: Standard vs Robust Ticket')
    ax.set_xticks(x)
    ax.set_xticklabels(feature_names, rotation=45, ha='right')
    ax.legend()
    ax.grid(True, alpha=0.3, axis='y')
    
    plt.tight_layout()
    plt.savefig('../figures/feature_importance_comparison.pdf', dpi=300, bbox_inches='tight')
    plt.show()

## Summary

Robust tickets preserve robust features (moneyness, time) more than non-robust features (return, P&L).